<a href="https://colab.research.google.com/github/kushal-arya/Deep-Learning-Projects/blob/main/Deeplearning_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from shutil import copyfile



In [ ]:
import os
from google.colab import drive
from tensorflow.keras.optimizers import RMSprop

drive.mount('/content/drive', force_remount=True)

# After mounting, you can use the path to your dataset
base_dir = '/content/drive/My Drive/kagglecatsanddogs_5340/PetImages'

Mounted at /content/drive


In [ ]:
import os
from PIL import Image
import shutil
import random

# Define the paths to 'Cat' and 'Dog' directories
cat_dir = os.path.join(base_dir, 'Cat')
dog_dir = os.path.join(base_dir, 'Dog')

# Define and create new directories for the training and validation sets
train_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Make new directories
os.makedirs(train_cats_dir, exist_ok=True)
os.makedirs(train_dogs_dir, exist_ok=True)
os.makedirs(validation_cats_dir, exist_ok=True)
os.makedirs(validation_dogs_dir, exist_ok=True)

# Function to split data into training and validation sets
def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = os.path.join(SOURCE, filename)  # Corrected path concatenation
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    validation_set = shuffled_set[training_length:]

    for filename in training_set:
        this_file = os.path.join(SOURCE, filename)  # Corrected path concatenation
        destination = os.path.join(TRAINING, filename)  # Corrected path concatenation
        shutil.copyfile(this_file, destination)

    for filename in validation_set:
        this_file = os.path.join(SOURCE, filename)  # Corrected path concatenation
        destination = os.path.join(VALIDATION, filename)  # Corrected path concatenation
        shutil.copyfile(this_file, destination)

# Split data into training and validation datasets
split_size = .8
split_data(cat_dir, train_cats_dir, validation_cats_dir, split_size)
split_data(dog_dir, train_dogs_dir, validation_dogs_dir, split_size)


In [ ]:
# Function to remove corrupted images
def remove_corrupted_images(directory):
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)
        if os.path.isdir(subdir_path):
            for file in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, file)
                if os.path.isfile(file_path):
                    try:
                        with Image.open(file_path) as img:
                            img.verify()
                    except (IOError, SyntaxError):
                        print('Bad file:', file_path)
                        os.remove(file_path)

# Remove corrupted images from training and validation sets
remove_corrupted_images(train_dir)
remove_corrupted_images(validation_dir)

# Create image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)




Bad file: /content/drive/My Drive/kagglecatsanddogs_5340/PetImages/training/cats/7022.jpg


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)




Found 17656 images belonging to 2 classes.
Found 9715 images belonging to 2 classes.


In [ ]:
# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),  # updated argument here
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # assumes 2000 training images
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50,  # assumes 1000 validation images
    verbose=2
)

Epoch 1/15
100/100 - 32s - loss: 0.6263 - accuracy: 0.6970 - val_loss: 0.5778 - val_accuracy: 0.7580 - 32s/epoch - 323ms/step
Epoch 2/15
100/100 - 10s - loss: 0.6057 - accuracy: 0.7145 - val_loss: 0.6160 - val_accuracy: 0.7000 - 10s/epoch - 99ms/step
Epoch 3/15
100/100 - 11s - loss: 0.6152 - accuracy: 0.6920 - val_loss: 0.5668 - val_accuracy: 0.7530 - 11s/epoch - 114ms/step
Epoch 4/15
100/100 - 12s - loss: 0.5899 - accuracy: 0.7150 - val_loss: 0.5314 - val_accuracy: 0.7500 - 12s/epoch - 118ms/step
Epoch 5/15
100/100 - 9s - loss: 0.5744 - accuracy: 0.7210 - val_loss: 0.5236 - val_accuracy: 0.7440 - 9s/epoch - 93ms/step
Epoch 6/15
100/100 - 12s - loss: 0.5356 - accuracy: 0.7575 - val_loss: 0.4921 - val_accuracy: 0.7720 - 12s/epoch - 122ms/step
Epoch 7/15
100/100 - 12s - loss: 0.5202 - accuracy: 0.7495 - val_loss: 0.5049 - val_accuracy: 0.7770 - 12s/epoch - 119ms/step
Epoch 8/15
100/100 - 12s - loss: 0.5138 - accuracy: 0.7575 - val_loss: 0.5161 - val_accuracy: 0.7570 - 12s/epoch - 124ms/s

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the InceptionV3 model pre-trained on ImageNet data
inception_v3_base = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(150, 150, 3)))




87910968/87910968 [==============================] - 3s 0us/step


In [ ]:
# Unfreeze the top 30 layers while leaving BatchNorm layers frozen
for layer in inception_v3_base.layers[:-30]:
    layer.trainable = False
for layer in inception_v3_base.layers[-30:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

# Adding custom layers on top of the base model
x = inception_v3_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)



In [ ]:
# Creating the final model
model = Model(inputs=inception_v3_base.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 74, 74, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 74, 74, 32)           96        ['conv2d_4[0][0]']            
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 74, 74, 32)           0         ['batch_normalization[0][0

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # This should be the number of batches per epoch, typically data_size / batch_size
    epochs=15,  # Reduced for quick testing, increase this for real training
    validation_data=validation_generator,
    validation_steps=50,  # This should be the number of validation batches, typically validation_data_size / batch_size
    verbose=2
)

# Print the accuracy
final_accuracy = history.history['accuracy'][-1]
final_val_accuracy = history.history['val_accuracy'][-1]
print(f"Final training accuracy: {final_accuracy*100:.2f}%")
print(f"Final validation accuracy: {final_val_accuracy*100:.2f}%")

Epoch 1/15
100/100 - 24s - loss: 0.4056 - accuracy: 0.8420 - val_loss: 0.1600 - val_accuracy: 0.9280 - 24s/epoch - 237ms/step
Epoch 2/15
100/100 - 12s - loss: 0.1809 - accuracy: 0.9495 - val_loss: 0.1266 - val_accuracy: 0.9470 - 12s/epoch - 117ms/step
Epoch 3/15
100/100 - 13s - loss: 0.1388 - accuracy: 0.9510 - val_loss: 0.0959 - val_accuracy: 0.9670 - 13s/epoch - 134ms/step
Epoch 4/15
100/100 - 12s - loss: 0.1222 - accuracy: 0.9565 - val_loss: 0.0867 - val_accuracy: 0.9690 - 12s/epoch - 122ms/step
Epoch 5/15
100/100 - 10s - loss: 0.1062 - accuracy: 0.9660 - val_loss: 0.0746 - val_accuracy: 0.9760 - 10s/epoch - 103ms/step
Epoch 6/15
100/100 - 13s - loss: 0.1168 - accuracy: 0.9560 - val_loss: 0.0790 - val_accuracy: 0.9750 - 13s/epoch - 132ms/step
Epoch 7/15
100/100 - 12s - loss: 0.1180 - accuracy: 0.9620 - val_loss: 0.0726 - val_accuracy: 0.9720 - 12s/epoch - 116ms/step
Epoch 8/15
100/100 - 12s - loss: 0.0888 - accuracy: 0.9700 - val_loss: 0.0617 - val_accuracy: 0.9770 - 12s/epoch - 115

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/drive/My Drive/my_inceptionv3_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the H5 file
loaded_model = load_model('/content/drive/My Drive/my_inceptionv3_model.h5')


# Pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import time
import copy


In [ ]:
# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),  # Increased size for InceptionV3
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(299),  # Increased size for InceptionV3
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
base_dir = '/content/drive/My Drive/kagglecatsanddogs_5340/PetImages'

# Define the paths to 'training' and 'validation' directories
train_dir = os.path.join(base_dir, 'training')
val_dir = os.path.join(base_dir, 'validation')
# Create datasets using ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, data_transforms['train']),
    'validation': datasets.ImageFolder(val_dir, data_transforms['validation'])
}

# Create data loaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=20, shuffle=True),
    'validation': DataLoader(image_datasets['validation'], batch_size=20, shuffle=False)
}


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

        # Dummy input to calculate flat features
        self._num_flat_features = self._get_conv_output((1, 3, 299, 299))

        # Define the fully connected layers
        self.fc1 = nn.Linear(self._num_flat_features, 512)
        self.fc2 = nn.Linear(512, 2)

    def _get_conv_output(self, shape):
        batch_size = shape[0]
        input = torch.rand(shape)
        output = self.pool(self.relu(self.conv1(input)))
        output = self.pool(self.relu(self.conv2(output)))
        output = self.pool(self.relu(self.conv3(output)))
        return output.view(batch_size, -1).size(1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, self._num_flat_features)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x




# Create the model instance
custom_cnn_model = CustomCNN()

# Move the model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_cnn_model.to(device)

# Loss function and optimizer
criterion_custom = nn.CrossEntropyLoss()
optimizer_custom = optim.Adam(custom_cnn_model.parameters(), lr=1e-4)



In [ ]:
def train_model_custom(model, criterion, optimizer, train_loader, val_loader, num_epochs=25, device='cpu'):
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            data_loader = train_loader if phase == 'train' else val_loader
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # Deep copy the model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc

    print('Best val Acc: {:4f}'.format(best_acc))
    return model


In [ ]:
# Move the model to the specified device
custom_cnn_model.to(device)

# Train the model using dataloaders['train'] and dataloaders['validation']
trained_model = train_model_custom(custom_cnn_model, criterion_custom, optimizer_custom, dataloaders['train'], dataloaders['validation'], num_epochs=10, device=device)


Epoch 1/10
----------


In [ ]:
# Load the pre-trained InceptionV3 model
inception = models.inception_v3(pretrained=True, aux_logits=True)
inception.aux_logits = False

# Unfreeze the last 30 layers
child_counter = 0
for child in inception.children():
    if child_counter > len(list(inception.children())) - 30:
        for param in child.parameters():
            param.requires_grad = True
    else:
        for param in child.parameters():
            param.requires_grad = False
    child_counter += 1

# Modify the classifier for binary classification
num_ftrs = inception.fc.in_features
inception.fc = nn.Linear(num_ftrs, 2)

# Move the model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, inception.parameters()), lr=1e-5)



In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=5, device='cpu'):
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))

        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            data_loader = train_loader if phase == 'train' else val_loader
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    # Handle the Inception model output based on the mode
                    if phase == 'train' and model.aux_logits:
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4 * loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # Save the model with best accuracy
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc

    print('Best val Acc: {:4f}'.format(best_acc))
    return model

# Assuming device, dataloaders, model, criterion, optimizer are correctly set up
model_ft = train_model(inception, criterion, optimizer, dataloaders['train'], dataloaders['validation'], num_epochs=5, device=device)


In [ ]:
from torchsummary import summary
summary(inception, (3, 299, 299))

In [ ]:
# Assuming `val_loader` is your DataLoader for validation data
def evaluate_pytorch_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_loss += loss.item() * inputs.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

val_loss_pt, val_accuracy_pt = evaluate_pytorch_model(model_ft, dataloaders['validation'])
print(f"PyTorch Validation Accuracy: {val_accuracy_pt*100:.2f}%")
print(f"PyTorch Validation Loss: {val_loss_pt:.4f}")


In [ ]:

# Save the entire model
torch.save(model_ft, '/content/drive/My Drive/pytorch_inception_model.pth')


In [ ]:
# Load the saved model
model_path = '/content/drive/My Drive/pytorch_inception_model.pth'
inception.load_state_dict(torch.load(model_path))
inception.to(device)


In [ ]:
print(f"TensorFlow vs PyTorch Validation Accuracy: {final_accuracy*100:.2f}% vs {val_accuracy_pt*100:.2f}%")
print(f"TensorFlow vs PyTorch Validation Loss: {final_val_accuracy:.4f} vs {val_loss_pt:.4f}")
